In [ ]:
using Distributed
using SharedArrays
addprocs(11);

OpenBLAS blas_thread_init: pthread_create failed for thread 1 of 8: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 2 of 8: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 3 of 8: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 4 of 8: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 5 of 8: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 6 of 8: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_th

In [ ]:
@everywhere include("inverse_prob.jl")
@everywhere using PyPlot
@everywhere using DSP

In [ ]:
@eval @everywhere begin
    Nx = 201;
    Ny = 201;
    h = 0.005;
    x = range(0,step=h,length=Nx)
    y = range(0,step=h,length=Ny)

    Fs = 600;
    dt = 1/Fs
    Nt = 1200;
    t = range(0, length=Nt, step=dt)

    source = source_ricker(10, 0.15, t)
    f = digitalfilter(Lowpass(2, fs = Fs), FIRWindow(hamming(61)))
    source = conv(source,f)
    source = source[31:end-30]
    source = source./ maximum(source)
    
    source_position = zeros(11,2)
    for i = 1:11
        source_position[i,:] = [1 20*(i-1)+1]
#         source_position[i,:] = [1 101]
    end
    source = repeat(source, 1, 11)

    receiver_position = zeros(51,2)
    for i = 1:51
        receiver_position[i,:] = [201, (i-1)*4+1]
    end
#     for i = 52:101
#         receiver_position[i,:] = [101, (i-51)*2+1]
#     end

    c0 = 1*ones(Nx, Ny)
    rho0 = ones(Nx, Ny)
    for i = 1:Nx
        for j = 1:Ny
            if sqrt((x[i]-0.5).^2 + (y[j]-0.5).^2) < 0.33
                c0[i,j] = 1.2
            end
        end
    end
#     c0[50:end,:] .= 1.2

    c = 1*ones(Nx, Ny)
#     c[1:end,:] = imfilter(c0[1:end,:], Kernel.gaussian(10));
    rho = ones(Nx, Ny);
end
figure
subplot(121)
imshow(c); colorbar(shrink=0.3); title("c"); 
scatter(source_position[:,2].-1, source_position[:,1].-1, alpha=0.5)
scatter(receiver_position[:,2].-1, receiver_position[:,1].-1, alpha=0.5)
subplot(122)
imshow(c0); colorbar(shrink=0.3); title("c0"); 
scatter(source_position[:,2].-1, source_position[:,1].-1, alpha=0.5)
scatter(receiver_position[:,2].-1, receiver_position[:,1].-1, alpha=0.5)
tight_layout()

In [ ]:
M = cost_matrix_1d(t,t; p=2)
reg = 1e-5
reg_m = 1e0
reg_p = 0.5
iterMax = 1000;

save_ratio = 10;
pml_len = 30;
pml_coef = 100;

In [ ]:
@time data, u = multi_solver_parallel(c, rho, Nx, Ny, Nt, h, dt, source, source_position, receiver_position; pml_len=pml_len, pml_coef=pml_coef, save_ratio=save_ratio);

@time data0, u0 = multi_solver_parallel(c0, rho0, Nx, Ny, Nt, h, dt, source, source_position, receiver_position; pml_len=pml_len, pml_coef=pml_coef, save_ratio=save_ratio);


In [ ]:
figure;
time_ind = 40
subplot(121); imshow(u[:,:,time_ind,1],cmap=ColorMap("gray")); colorbar(shrink=0.4); clim(-0.1,0.1)
subplot(122); imshow(u0[:,:,time_ind,1],cmap=ColorMap("gray")); colorbar(shrink=0.4); clim(-0.1,0.1)
tight_layout()

In [ ]:
figure;
subplot(121); imshow(data[:,:,2],cmap=ColorMap("gray"), aspect=0.1); colorbar(shrink=0.4); clim([-0.01,0.01])
xlabel("receiver"); ylabel("time (s)")
subplot(122); imshow(data0[:,:,2],cmap=ColorMap("gray"), aspect=0.1); colorbar(shrink=0.4); clim([-0.01,0.01])
xlabel("receiver"); ylabel("time (s)")
tight_layout()

In [ ]:
op_fn(x) = eval_fn_l2(data0, x, rho, Nx, Ny, Nt, h, dt, source, source_position, receiver_position; save_ratio=save_ratio, pml_len=pml_len, pml_coef=pml_coef)
op_grad(x) = eval_grad_l2(data0, x, rho, Nx, Ny, Nt, h, dt, source, source_position, receiver_position; save_ratio=save_ratio, pml_len=pml_len, pml_coef=pml_coef);

# @time xx,gg = op_grad(c[:])
# @time xx1 = op_fn(c[:])

# imshow(reshape(gg, Nx, Ny)); colorbar()

In [9]:
# aaa = update_fn(c, 6e-6, gg, 1, 1.2)
# imshow(reshape(aaa, Nx, Ny)); colorbar(); clim(1,1.2)

In [10]:
# @time data11, u11 = multi_solver_parallel(aaa, rho, Nx, Ny, Nt, h, dt, source, source_position, receiver_position; pml_len=pml_len, pml_coef=pml_coef, save_ratio=save_ratio);
# 0.5 * norm(data11 - data0,2) ^ 2

In [11]:
# 0.5 * norm(data - data0,2) ^ 2

In [12]:
# xx_gd, ff_gd = gradient_descent(op_fn, op_grad, c[:], 1e-5, 4, 1, 1.1; rho=0.5, c=1e-10, maxSearchTime=10, threshold=1e-5);

In [13]:
# matwrite("FWI_1_L2_GD.mat", Dict("xx" => xx_gd,"ff" => ff_gd,"c"=>c,"c0"=>c0); compress = false)

In [14]:
# imshow(reshape(xx_gd,Nx,Ny)); colorbar(shrink=0.3)

In [15]:
xx_ll, ff_ll = LBFGS(op_fn, op_grad, c[:], 6e-6, 3, 1, 1.2; m=5, alpha_search=1, rho=0.9, c=1e-10, maxSearchTime=5, threshold=1e-10);

Main iteration: 1
Start line search. fk: 3.31077e+01
    alpha: 6.00000e-06    fk1: 7.21472e+00    fk-c*alpha*gradk^2: 3.31031e+01
Line search succeed. Search time: 0.
----------------------------------------------------------------
Start LBFGS.
----------------------------------------------------------------
Main iteration: 2
Start line search. fk: 7.21472e+00
    alpha: 1.00000e+00    fk1: 3.70450e+01    fk-c*alpha*gradk^2: 7.21472e+00
    alpha: 9.00000e-01    fk1: 3.53549e+01    fk-c*alpha*gradk^2: 7.21472e+00
    alpha: 8.10000e-01    fk1: 3.35830e+01    fk-c*alpha*gradk^2: 7.21472e+00
    alpha: 7.29000e-01    fk1: 3.17368e+01    fk-c*alpha*gradk^2: 7.21472e+00
    alpha: 6.56100e-01    fk1: 2.98239e+01    fk-c*alpha*gradk^2: 7.21472e+00
    alpha: 5.90490e-01    fk1: 2.78945e+01    fk-c*alpha*gradk^2: 7.21472e+00
Line search failed. Search time: 5. Try to decrease search coef c.
----------------------------------------------------------------
Line search Failed. Try decrease lin

In [ ]:
imshow(reshape(xx_ll,Nx,Ny)); colorbar(shrink=0.3)